In [ ]:
import pandas as pd
import numpy as np

# 假设 axisfuzzy 库已经安装并且在 Python 路径中
# 如果在本地运行，需要确保 axisfuzzy 目录在 PYTHONPATH 中
from axisfuzzy.analysis.pipeline import FuzzyPipeline
from axisfuzzy.analysis.components.basic import (
    NormalizationTool,
    SimpleAggregationTool,
    WeightNormalizationTool,
    StatisticsTool
)
from axisfuzzy.analysis.components.base import AnalysisComponent
from axisfuzzy.analysis.contracts import contract, WeightVector, CrispTable, NormalizedWeights

In [ ]:
# --- 0. 定义一个用于合并和加权的自定义组件 ---
class WeightedSumTool(AnalysisComponent):
    """一个自定义组件，用于合并分数并计算加权和。"""
    @contract(
        inputs={'scores_a': 'WeightVector', 'scores_b': 'WeightVector', 'weights': 'NormalizedWeights'},
        outputs={'final_scores': 'WeightVector', 'combined_data': 'CrispTable'}
    )
    def run(self, scores_a: pd.Series, scores_b: pd.Series, weights: pd.Series):
        # 合并分数到一个 DataFrame
        combined_df = pd.DataFrame({'objective': scores_a, 'subjective': scores_b})
        # 计算加权和
        final_scores = (combined_df * weights).sum(axis=1)
        return {'final_scores': final_scores, 'combined_data': combined_df}

In [ ]:
# --- 1. 创建示例数据 ---
performance_data = pd.DataFrame({
    'Cost': [100, 120, 95, 110],       # 成本越低越好
    'Quality': [98, 95, 99, 96],      # 质量分越高越好
    'DeliverySpeed': [5, 7, 4, 6]     # 交付天数越少越好
}, index=['Supplier_A', 'Supplier_B', 'Supplier_C', 'Supplier_D'])

survey_data = pd.DataFrame({
    'ServiceAttitude': [8, 7, 9, 8],  # 服务态度分
    'Innovation': [7, 9, 8, 7]        # 创新能力分
}, index=['Supplier_A', 'Supplier_B', 'Supplier_C', 'Supplier_D'])

# 注意：为了简单起见，我们对成本和交付速度进行预处理，使其变为正向指标
performance_data['Cost'] = 1 / performance_data['Cost']
performance_data['DeliverySpeed'] = 1 / performance_data['DeliverySpeed']

print("--- 初始性能数据 (预处理后) ---")
print(performance_data)
print("\n--- 初始问卷数据 ---")
print(survey_data)

In [ ]:
# --- 2. 定义子流程 (Pipelines) ---

# 子流程 1: 客观数据分析
objective_pipeline = FuzzyPipeline(name="ObjectiveAnalysis")
obj_input = objective_pipeline.input("perf_data", contract='CrispTable')
# 使用 max-normalization
obj_normalizer = NormalizationTool(method='max', axis=0)
obj_aggregator = SimpleAggregationTool(operation='mean', axis=1)
normalized_perf = objective_pipeline.add(obj_normalizer.run, data=obj_input)
objective_scores = objective_pipeline.add(obj_aggregator.run, data=normalized_perf)

# 子流程 2: 主观数据分析
subjective_pipeline = FuzzyPipeline(name="SubjectiveAnalysis")
subj_input = subjective_pipeline.input("survey_data", contract='CrispTable')
# 使用 min-max normalization
subj_normalizer = NormalizationTool(method='min_max', axis=0)
subj_aggregator = SimpleAggregationTool(operation='mean', axis=1)
normalized_survey = subjective_pipeline.add(subj_normalizer.run, data=subj_input)
subjective_scores = subjective_pipeline.add(subj_aggregator.run, data=normalized_survey)

In [ ]:
# --- 3. 定义主评估流程 ---
main_pipeline = FuzzyPipeline(name="MainEvaluation")

# 定义主流程的两个输入
perf_input = main_pipeline.input("performance_data", contract='CrispTable')
survey_input = main_pipeline.input("survey_data", contract='CrispTable')

# 新增：为 initial_weights 创建一个输入节点
# 这样 Pipeline 引擎就知道 'initial_weights_input' 是一个数据输入
# 它的契约是 WeightVector
initial_weights_input = main_pipeline.input("initial_weights_val", contract='WeightVector')

# 并行执行两个子流程
# 注意：kwargs 的键 ('perf_data', 'survey_data') 必须匹配子流程中定义的输入名
objective_result = main_pipeline.add(objective_pipeline, perf_data=perf_input)
subjective_result = main_pipeline.add(subjective_pipeline, survey_data=survey_input)

# 定义权重
# 这是一个静态输入，不是来自上一步，所以直接作为值传入
initial_weights = pd.Series({'objective': 0.6, 'subjective': 0.4})

# 实例化组件
weight_normalizer = WeightNormalizationTool()
combiner = WeightedSumTool()
stats_tool = StatisticsTool()

# 归一化权重
normalized_weights = main_pipeline.add(weight_normalizer.run, weights=initial_weights_input)

# 合并分数并计算加权总分
combined_results = main_pipeline.add(
    combiner.run,
    scores_a=objective_result,
    scores_b=subjective_result,
    weights=normalized_weights
)

# 对最终分数进行统计分析
# combined_results 是一个字典，包含 'final_scores' 和 'combined_data'
# 我们只对 final_scores 感兴趣
final_statistics = main_pipeline.add(
    stats_tool.run,
    data=combined_results['combined_data'] # 这里我们用合并后的数据做统计
)

In [ ]:
print("\n--- 可视化工作流 ---")
try:
    # 优先使用 Graphviz 直接在 Jupyter 中显示
    print("尝试使用 'graphviz' 引擎直接显示...")
    # 将 visualize 的返回值直接打印或作为 Cell 的最后一行，Jupyter 会自动渲染
    display(main_pipeline.visualize(engine='graphviz', format='svg')) # 使用 display 函数确保显示
except (ImportError, Exception) as e:
    print(f"Graphviz 引擎不可用 ({e}). 回退到 'matplotlib' 引擎...")
    try:
        # 如果 Graphviz 失败，则使用 Matplotlib 直接在 Jupyter 中显示
        print("尝试使用 'matplotlib' 引擎直接显示...")
        display(main_pipeline.visualize(engine='matplotlib')) # Matplotlib 默认会调用 plt.show()
    except ImportError as e_mpl:
        print(f"Matplotlib 引擎也不可用: {e_mpl}. 无法生成可视化图。")

In [ ]:
# --- 5. 执行主 Pipeline 并打印结果 ---
print("\n--- 执行 Pipeline ---")
final_outputs = main_pipeline.run(
    initial_data={
        'performance_data': performance_data,
        'survey_data': survey_data,
        'initial_weights_val': initial_weights
    },
    return_intermediate=False # 我们只关心最终结果
)

print("\n--- Pipeline 执行最终结果 ---")
# 主管道有两个末端节点: WeightedSumTool 和 StatisticsTool
# 所以 final_outputs 是一个字典
for tool_name, result in final_outputs.items():
    print(f"\n>> 来自工具 '{tool_name}' 的输出:")
    if isinstance(result, dict):
        for output_name, data in result.items():
            print(f"  - {output_name}:")
            print(data)
    else:
        print(result)